In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import cross_validation
%pylab inline

In [ ]:
w = pd.read_csv('tmp/engineered_features.csv', index_col='file')

In [ ]:
tr = w[pd.notnull(w.sponsored)]
tr = tr.fillna(0)
#tr.info()

In [ ]:
trl = tr.sponsored
del tr['sponsored']

In [ ]:
te = w[pd.isnull(w.sponsored)]
del te['sponsored']
te = te.fillna(0)
#te.info()

In [ ]:
m = RandomForestClassifier(n_estimators=10, n_jobs=-1)
#m = KNeighborsClassifier(n_neighbors=5)
#m = SVC(probability=True)

In [ ]:
scores = cross_validation.cross_val_score(m, tr, trl, cv=3, scoring='roc_auc')

In [ ]:
# RandomForestClassifier(n_estimators=500, max_depth=30, oob_score=True, n_jobs=-1):
#     array([ 0.90357458,  0.90298073,  0.90189454])
# KNeighborsClassifier(n_neighbors=5):
#     array([ 0.67784316,  0.68113898,  0.6737584 ])
scores

In [ ]:
#m.fit(tr, trl)

In [ ]:
# preds = DataFrame(m.predict_proba(te))[1]
# sub = DataFrame({'file': te.index, 'sponsored': preds})
# sub.info()

In [ ]:
# sub.to_csv('output/prec_rec_count_randfor_02.csv', index=False)